In [1]:
%reset -f
%matplotlib inline


import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import time

print(cv.__version__)

3.4.2


## Projeto 2-1: Estabilização de imagens

**Motivação**: https://www.youtube.com/watch?v=4vt7bGEen2s

O projeto de estabilização envolve entender o fluxo ótico e utilizar ao seu favor para translacionar sua imagem e garantir que está compensando eventuais oscilações da câmera

### Etapa 1
###### Encontro dos pontos notáveis
Utiliza a função Good Features to track

In [3]:
#https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html
captura = cv.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)

dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))


ret, old_frame = captura.read()

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **dt_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)


while(1):
    ret,frame = captura.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv.add(frame,mask)
    cv.imshow('frame',frame)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

    
captura.release()
cv.destroyAllWindows()


### Etapa 2
###### Translação da imagem para estabilização
Com os pontos notáveis da etapa anterior, faz a média dos pontos nos eixos X e Y entre a imagem anterior e a atual, e fica armazenda, para conseguir fazer a translação necessária e manter a filmagem estabilizada. Este método não é o ideal, já que a função que encontra os pontos, gera todos os pontos que a camera fornece, então a média acaba sendo afetada por isso.

In [4]:
#https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html
captura = cv.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)

dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))


ret, old_frame = captura.read()

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **dt_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)


while(1):
    ret,frame = captura.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    x=0
    y=0
    avg_x=0
    avg_y=0
    # draw the tracks
    for i in range(len(good_new)):
        x+=good_new[i][0]-good_old[i][0]
        y+=good_new[i][1]-good_old[i][1]
    avg_x+=x/(len(good_new))
    avg_y+=y/(len(good_new))
    
    #print(avg_x,avg_y)
    
    #transladando a imagem
    rows, cols, coisa = frame.shape
    M = np.array([[1, 0, -avg_x], [0, 1, -avg_y]], dtype=np.float32)
    img_shifted = cv.warpAffine(frame, M, (cols,rows))  # Terceiro argumento é o tamanho da imagem resultante.
    
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        img_shifted = cv.circle(img_shifted,(a,b),5,color[i].tolist(),-1)
    img = cv.add(img_shifted,mask)
    
    

    
    
    cv.imshow('frame',img_shifted)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

    
captura.release()
cv.destroyAllWindows()


### Etapa 3
###### Utilizando dense flow
Ao substituir o método anterior pelo dense flow, melhoramos o problema comentado na última etapa, já que o dense flow permite selecionar a área desejada para fazer o optical flow dos pontos. Seleciona a área central da imagem, para focar no rosto da pessoa, e a própria função retorna os pontos X e Y, bastando calcular a média e acumular para utilizar o método de translação.

In [5]:
#https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html
captura = cv.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)

dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))


ret, old_frame = captura.read()

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **dt_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

avg_x=0
avg_y=0
while(1):
    ret,frame = captura.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    #calculate dense optical flow
    flow = cv.calcOpticalFlowFarneback(old_gray[200:280, 280:360],frame_gray[200:280, 280:360], None, 0.5, 3, 15, 3, 5, 1.2, 0)
       
    avg_x+=np.average(flow[:, :,0])
    avg_y+=np.average(flow[:, :,1])
    
    #transladando a imagem
    rows, cols, coisa = frame.shape
    M = np.array([[1, 0, -avg_x], [0, 1, -avg_y]], dtype=np.float32)
    img_shifted = cv.warpAffine(frame, M, (cols,rows))  # Terceiro argumento é o tamanho da imagem resultante.
    
  

    
    
    cv.imshow('frame',img_shifted)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = flow.reshape(-1,1,2)

    
captura.release()
cv.destroyAllWindows()


### Etapa Teste
###### Eliminando as bordas
Para a eliminação das bordas, foi testado a utilização da funçao resize do opencv, porém, a tentativa de fazer o corte gerou uma falha devido a falta de proporção da imagem gerada. O que deve ser feito para a solução deste problema é verificar se a resolução resultante está porporcional, caso não esteja, devemos fazer um corte extra no sentido em que temos um excesso, assim, mantemos a proporção sem perder muita imagem

In [7]:
#https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html
#https://stackoverflow.com/questions/4195453/how-to-resize-an-image-with-opencv2-0-and-python2-6

captura = cv.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)

dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))


ret, old_frame = captura.read()

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **dt_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

avg_x=0
avg_y=0
while(1):
    ret,frame = captura.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    #calculate dense optical flow
    flow = cv.calcOpticalFlowFarneback(old_gray[200:280, 280:360],frame_gray[200:280, 280:360], None, 0.5, 3, 15, 3, 5, 1.2, 0)
       
    avg_x+=np.average(flow[:, :,0])
    avg_y+=np.average(flow[:, :,1])

    #transladando a imagem
    rows, cols, coisa = frame.shape
    M = np.array([[1, 0, -avg_x], [0, 1, -avg_y]], dtype=np.float32)
    img_shifted = cv.warpAffine(frame, M, (cols,rows))  # Terceiro argumento é o tamanho da imagem resultante.
    
    #image scaling
    height, width = img_shifted.shape[:2]
    max_height = 640
    max_width = 480
    new_y=int(height-avg_y)
    new_x=int(width-avg_x)

    img_scale = cv.resize(img_shifted,(new_y,new_x)) 
    
    
    cv.imshow('frame',img_scale)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = flow.reshape(-1,1,2)

    
captura.release()
cv.destroyAllWindows()
